# Disclaimer & Copyright

Copyright 2024 Forusone : hangsik@google.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Llamaindex on Vertex AI
* This notebook explains generating metadata based on contents of a pdf file.
* The pdf file could be a small chunk file which has 2~3 pages.
* The metadata contains "Summary", "Questions", "Keywords" and original PDF file name.
* RAG system uses the metadata to retrieve the relevant context.


# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [2]:
%pip install --upgrade --quiet google-cloud-aiplatform


In [3]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko
 * Mount to the google drive to access the .ipynb files in the repository.


In [4]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [5]:
MODEL_NAME="gemini-1.5-flash-001"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization


In [6]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)


Image encoding function  
The encoded binary will be ingested as one of the contexts for LLM.

In [8]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai

# Create a RAG Corpus, Import Files, and Generate a response

# TODO(developer): Update and un-comment below lines
display_name = "Galaxy manual"

# Initialize Vertex AI API once per session
vertexai.init(project=PROJECT_ID, location="us-central1")

# Create RagCorpus
# Configure embedding model, for example "text-embedding-004".
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-multilingual-embedding-002"
)

rag_corpus = rag.create_corpus(
    display_name=display_name,
    embedding_model_config=embedding_model_config,
)

paths = ["gs://it_laws_kr/law_pdf"]



# Import Files to the RagCorpus
response = rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size=512,  # Optional
    chunk_overlap=100,  # Optional
    max_embedding_requests_per_min=900,  # Optional
)


In [9]:

import time

t1 = time.perf_counter()

# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],

    text="신용정보 보호법에서 개인정보에 대한 내용",
    similarity_top_k=10,  # Optional
    vector_distance_threshold=0.3,  # Optional
)

t2 = time.perf_counter()

print(f"elapsed time : {t2 - t1}")

for context in response.contexts.contexts:
    print(context)

    print(context.text)
    print(context.distance)

    print('=====')


# print(response.contexts.contexts)

elapsed time : 2.1995783020000204
source_uri: "gs://it_laws_kr/law_pdf/\341\204\211\341\205\265\341\206\253\341\204\213\341\205\255\341\206\274\341\204\214\341\205\245\341\206\274\341\204\207\341\205\251\341\204\213\341\205\264 \341\204\213\341\205\265\341\204\213\341\205\255\341\206\274 \341\204\206\341\205\265\341\206\276 \341\204\207\341\205\251\341\204\222\341\205\251\341\204\213\341\205\246 \341\204\200\341\205\252\341\206\253\341\204\222\341\205\241\341\206\253 \341\204\207\341\205\245\341\206\270\341\204\205\341\205\262\341\206\257(\341\204\207\341\205\245\341\206\270\341\204\205\341\205\262\341\206\257)(\341\204\214\341\205\24619234\341\204\222\341\205\251)(20230915).pdf"
text: "1. \352\260\234\354\235\270\354\213\240\354\232\251\354\240\225\353\263\264 \353\263\264\355\230\270 \353\260\217 \352\264\200\353\246\254\354\227\220 \352\264\200\355\225\234 \352\270\260\353\263\270\352\263\204\355\232\215(\354\264\235\354\236\220\354\202\260, \354\242\205\354\227\205\354\233\220 \354